# Imports

In [ ]:
import os
from PIL import Image
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
import numpy as np

# Discriminator

In [ ]:
def unconditional_discriminator(in_shape=(28,28,1), n_classes=4):
    model = Sequential()
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    # downsample
    model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # classifier
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Generator

In [ ]:
def unconditional_generator(latent_dim):
    model = Sequential()
    # foundation for 7x7 image
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    # upsample to 14x14
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 28x28
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # generate
    model.add(Conv2D(1, (7,7), activation='tanh', padding='same'))
    return model

# Conditional GAN

In [ ]:
def unconditional_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# Load train/test datasets

In [ ]:
def load_data():
    '''
    Load images in grayscale format with size 28x28 and assign class label depending on the folder they are in 
        
        Image object  |   Class
                      |
            hotel     |     0
          mountains   |     1 
            sea       |     2
           trees      |     3
    '''
    
    traindata_path = './train'
    trainX = []
    trainy = []
    for label_id, folder in enumerate(os.listdir(traindata_path)):
        for image_name in os.listdir('{}/{}'.format(traindata_path, folder)):
            if os.path.isfile('{}/{}/{}'.format(traindata_path, folder, image_name)):
                image = Image.open('{}/{}/{}'.format(traindata_path, folder, image_name)).convert('LA')
                image = image.resize((28,28))
                trainX.append(np.array(image)[:,:,0])
                trainy.append(label_id)

    testdata_path = './test'
    testX = []
    testy = []
    for label_id, folder in enumerate(os.listdir(testdata_path)):
        for image_name in os.listdir('{}/{}'.format(testdata_path, folder)):
            if os.path.isfile('{}/{}/{}'.format(testdata_path, folder, image_name)):
                image = Image.open('{}/{}/{}'.format(testdata_path, folder, image_name)).convert('LA')
                image = image.resize((28,28))
                testX.append(np.array(image)[:,:,0])
                testy.append(label_id)

                
    trainX = np.array(trainX)
    trainy = np.array(trainy)
    testX = np.array(testX)
    testy = np.array(testy)
    
    # summarize the shape of the dataset
    print('Train', trainX.shape, trainy.shape)
    print('Test', testX.shape, testy.shape)

    return (trainX, trainy), (testX, testy)

def load_real_samples():
    # load dataset
    (trainX, _), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return X

# Generate real / fake samples

In [ ]:
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # select images
    X = dataset[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return X, y

# generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = generator.predict(x_input)
    # create class labels
    y = zeros((n_samples, 1))
    return X, y

# Generate latent points

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# Train

In [ ]:
def train_unconditional_gan(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch(X_real, y_real)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                (i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
    # save the generator model
    g_model.save('ucgan_generator_2000epochs_128bs.h5')
    d_model.save('ucgan_discriminator_2000epochs_128bs.h5')

# Main

In [ ]:
# size of the latent space
latent_dim = 100
# create the discriminator
discriminator = unconditional_discriminator()
# create the generator
generator = unconditional_generator(latent_dim)
# create the gan
gan_model = unconditional_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
# train model
train_unconditional_gan(generator, discriminator, gan_model, dataset, latent_dim)